In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import imageio
from wrappers.caiman_wrapper import *
from wrappers.dummy_wrapper import *
from wrappers.data_wrapper import *

### dummy

In [ ]:
file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
image = ImageData(file_path, 'dummy')

In [ ]:
dummy_image2image(ImageData(file_path, ''), opts={'a': 1})

### caiman

In [ ]:
info = {}
file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')
info['initial'] = {'file': ImageData(file_path, '')}

In [ ]:
info['caiman_mc'] = caiman_mc(*info['initial'].values())
info['cnmf'] = caiman_cnmf(*info['caiman_mc'].values())

In [ ]:
plt.imshow(info['caiman_mc']['images'].data[0])

### suite2p

In [ ]:
import os
from suite2p import default_ops
from wrappers.data_wrapper import *
from wrappers.suite2p_wrapper import *
from wrappers.args_check import args_check

In [ ]:


@args_check
def suite2p_registration(image: ImageData, opts: dict=None):
    import numpy as np
    from suite2p import registration
    from suite2p import default_ops

    ######### REGISTRATION #########
    refImg = image.data
    if len(refImg.shape) == 3:
        refImg = refImg[0]

    if opts is None:
        ops = {**default_ops()}
    else:
        ops = opts

    ops = registration.register_binary(ops, refImg=refImg) # register binary

    # compute metrics for registration
    if ops.get('do_regmetrics', True) and ops['nframes']>=1500:
        ops = registration.get_pc_metrics(ops)

    ops['images'] = ops['refImg'].astype(np.uint8)
    ops['meanImgE'] = ops['meanImgE']

    return ops

In [ ]:
info = {}

file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')
info['initial'] = {'file': ImageData(file_path, '')}

In [ ]:
info['file_convert'] = suite2p_file_convert(*info['initial'].values())
info['registration'] = suite2p_registration(*info['file_convert'].values())

In [ ]:
# info['run_s2p'] = run_s2p(file_path)
# info['roi'] = roi(info['run_s2p']['save_path'], info['run_s2p']['Lx'], info['run_s2p']['Ly'])

In [ ]:
# plt.figure(figsize=(16, 4))

# plt.subplot(1, 4, 1)
# plt.imshow(info['run_s2p']['max_proj'], cmap='gray')
# plt.title("Registered Image, Max Projection")

# plt.subplot(1, 4, 2)
# plt.imshow(np.nanmax(info['roi']['im'], axis=0), cmap='jet')
# plt.title("All ROIs Found")

# plt.subplot(1, 4, 3)
# plt.imshow(np.nanmax(info['roi']['im'][~info['roi']['iscell']], axis=0, ), cmap='jet')
# plt.title("All Non-Cell ROIs")

# plt.subplot(1, 4, 4)
# plt.imshow(np.nanmax(info['roi']['im'][info['roi']['iscell']], axis=0), cmap='jet')
# plt.title("All Cell ROIs");